# Convolutional Neural Network

### Importing the libraries

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

###  Define data transformations

In [ ]:
transform = transforms.Compose([transforms.Resize((64, 64)),
                                transforms.ToTensor(),  # Convert images to PyTorch tensors
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) ]) # Normalize pixel values

## Data Preprocessing

### Load training and validation data

In [ ]:
train_set = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/Projects/Image Classification/Lithium/Training_set', transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=16, shuffle=True, num_workers=2)

val_set = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/Projects/Image Classification/Lithium/Test_set', transform=transform)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=16, shuffle=False, num_workers=2)

# Building the CNN

### Define the CNN model for binary classification

In [ ]:
import torch.nn.functional as F

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)  # Adjusted input size for the first fully connected layer
        self.fc2 = nn.Linear(128, 2)  # 2 classes for binary classification: Anode and Cathode

    def forward(self, x):
       x = self.pool(F.relu(self.conv1(x)))
       x = self.pool(F.relu(self.conv2(x)))
       print("Size before flattening:", x.size())  # Print size before flattening
       x = x.view(-1, 64 * 16 * 16)  # Adjusted size for flattening
       x = F.relu(self.fc1(x))
       x = self.fc2(x)
       return x


### Instantiate the model, loss function, and optimizer

In [ ]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the CNN

In [ ]:
# Pass a dummy input through the first convolutional layer
conv1_output = model.conv1(torch.randn(1, 3, 64, 64))
print("Size after conv1:", conv1_output.size())

# Pass a dummy input through the second convolutional layer
conv2_output = model.conv2(conv1_output)
print("Size after conv2:", conv2_output.size())

Size after conv1: torch.Size([1, 64, 64, 64])
Size after conv2: torch.Size([1, 64, 64, 64])


In [ ]:
for epoch in range(25):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

In [ ]:
torch.save(model.state_dict(), 'classification_model.pth')

# Make Predictions

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in val_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the validation images: %d %%' % (
    100 * correct / total))

Size before flattening: torch.Size([16, 64, 16, 16])
Size before flattening: torch.Size([16, 64, 16, 16])
Size before flattening: torch.Size([16, 64, 16, 16])
Size before flattening: torch.Size([16, 64, 16, 16])
Size before flattening: torch.Size([16, 64, 16, 16])
Size before flattening: torch.Size([16, 64, 16, 16])
Size before flattening: torch.Size([16, 64, 16, 16])
Size before flattening: torch.Size([16, 64, 16, 16])
Size before flattening: torch.Size([16, 64, 16, 16])
Size before flattening: torch.Size([1, 64, 16, 16])
Accuracy of the network on the validation images: 80 %
